## Neutron data case

Tested with CIL version 21.0.0 on Linux

Download the neutron data from
https://bit.ly/2JnNRWc
and save and unzip zip-file in same directory as this notebook.

In [ ]:
# All CIL components
from cil.framework import ImageGeometry, ImageData
from cil.framework import AcquisitionGeometry, AcquisitionData

from cil.optimisation.algorithms import SIRT, CGLS, PDHG, GD
from cil.optimisation.operators import GradientOperator, BlockOperator
from cil.optimisation.functions import IndicatorBox, MixedL21Norm, \
                                       L2NormSquared, BlockFunction, \
                                       ZeroFunction, SmoothMixedL21Norm, \
                                       OperatorCompositionFunction

from cil.utilities.display import show2D

# CIL ASTRA plugin
from cil.plugins.astra.processors import FBP
from cil.plugins.astra.operators import  ProjectionOperator

# CIL Processors
from cil.processors import Slicer
from cil.processors import MaskGenerator
from cil.processors import Masker
from cil.processors import TransmissionAbsorptionConverter

# CIL IO
from cil.io import TIFFStackReader

# All other imports
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Pick slice
slice_no = 300

In [ ]:
# Data is not in a standardised format. Load in angles from text file in format provided.
angles_file = open('imat_rod_phantom_white_beam/golden_ratio_angles.txt', 'r') 

angles = []
for angle in angles_file:
    angles.append(float(angle.strip('0')))
angles_file.close()

n_angles = 186
angles = np.array(angles[:n_angles], dtype=np.float32)

In [ ]:
# Load stack of tiffs with TIFFStackReader into NumPy array
path = 'imat_rod_phantom_white_beam/'
reader = TIFFStackReader(file_name=path)
data = reader.read()

In [ ]:
# Display single slice sinogram
plt.figure()
plt.imshow(data[:, slice_no, :],cmap='inferno')
plt.colorbar()
plt.show()

In [ ]:
# Display single projection
plt.figure()
plt.imshow(data[0, :, :], cmap='inferno')
plt.colorbar()
plt.show()

In [ ]:
# Set up geometries
pixel_num_h = data.shape[2]
pixel_num_v = data.shape[1]

In [ ]:
ag = AcquisitionGeometry.create_Parallel3D().set_panel([pixel_num_v, pixel_num_h]).set_angles(angles=angles)

In [ ]:
# Create the Acquisition data from the data and geometry and permute for ASTRA ProjectionOperator
ad = AcquisitionData(geometry=ag, array=data)
ad.reorder(order='astra')

In [ ]:
# Data is not centered. Here apply simple centering by cropping cropping data by the centre of rotation offset>
ad = Slicer(roi = {'horizontal': (52, None)})(ad)
ag = ad.geometry

In [ ]:
# Set up mask for masking out pixels outside the interbal 1e-6 -- 1.
mask_generator = MaskGenerator.threshold(min_val=1e-6,max_val=1)
mask_generator.input = ad
mask = mask_generator.process()

In [ ]:
# Interpolate over masked pixels
masker = Masker(mask=mask, mode='interpolate', axis='horizontal', method='linear')
ad = masker(ad)

In [ ]:
# Air region in background should be ca 1.0 if properly normalised. Estimate scalar for background 
# value and use to normalise data.
rowtonorm = ad.subset(horizontal=20,force=True)
scale = rowtonorm.sum() / rowtonorm.size
print(scale)

In [ ]:
ad = ad / scale

In [ ]:
# Apply Lambert-Beer negative log-tranformation
converter = TransmissionAbsorptionConverter()
ad = converter(ad)

In [ ]:
# Display a sinogram slice
plt.figure()
show2D(ad.get_slice(vertical=slice_no),cmap='inferno')
plt.show()

In [ ]:
# Set default image geometry
ig = ag.get_ImageGeometry()

In [ ]:
# Simple FBP reconstruction, 3D volume
recon_fbp = FBP(ig, ag, device = 'gpu')(ad)

In [ ]:
# Show horizontal slice of reconstruction
plt.figure()
show2D(recon_fbp.get_slice(vertical=slice_no),cmap='inferno')
plt.show()

In [ ]:
# Extract single 2D dataset for subsequent experiments
ad2d = ad.get_slice(vertical=slice_no)

In [ ]:
# Create associated default 2D ImageGeometry
ig2d = ad2d.geometry.get_ImageGeometry()

In [ ]:
# Simple 2D FBP reconstruction
recon_fbp2d = FBP(ig2d, ad2d.geometry, device = 'gpu')(ad2d)

In [ ]:
# Display 2D FBP reconstruction and mark line for line profiles plots
vmin = -0.002
vmax =  0.012

yline = 89

plt.figure()
plt.imshow(recon_fbp2d.as_array(),vmin=vmin,vmax=vmax,cmap='inferno')
plt.colorbar()
plt.plot((0,459),(yline,yline),'-r')

In [ ]:
# Initial guess for iterative methods
x0 = ig2d.allocate()

In [ ]:
# 2D Forward operator
A2d = ProjectionOperator(ig2d, ad2d.geometry, 'gpu')

In [ ]:
# Set up CGLS algorithm
cgls2d = CGLS(x_init=x0, operator=A2d, data=ad2d, tolerance=0, max_iteration=1000)

In [ ]:
# Run specified number of iterations, with verbose printing
cgls2d.run(15, verbose=1)

In [ ]:
plt.figure()
show2D(cgls2d.solution, fix_range=(vmin,vmax), cmap='inferno')

In [ ]:
# Set up TV regularisation

# Define Gradient Operator and BlockOperator 
Grad = GradientOperator(ig2d)
K = BlockOperator(Grad,A2d)

# Define BlockFunction F using the MixedL21Norm() and the L2NormSquared()
alpha = 1.0
f1 =  alpha * MixedL21Norm()
f2 = 0.5 * L2NormSquared(b=ad2d)
F = BlockFunction(f1,f2)

# Define Function G simply as zero
G = ZeroFunction()

In [ ]:
# Compute operator norm and choose step-size sigma and tau such that sigma*tau||K||^{2}<1
normK =  K.norm()
sigma = 1
tau = 1/(sigma*normK**2)

In [ ]:
# Set up PDHG Algorithm
pdhg = PDHG(f=F, g=G, operator=K, tau=tau, sigma=sigma, 
            max_iteration=100000, update_objective_interval=10)

In [ ]:
# Run algorithm with extra verbose printing
pdhg.run(30000, verbose=2)

In [ ]:
plt.figure()
show2D(pdhg.solution, cmap='inferno',fix_range=(vmin,vmax))

In [ ]:
# Set up Smothed TV

# Smoothing parameter
epsilon = 1e-6

# Smooth TV functional
Grad = GradientOperator(ig2d)
f1gd = OperatorCompositionFunction(alpha*SmoothMixedL21Norm(epsilon), Grad)

# Least squares from basic building blocks
f2gd = OperatorCompositionFunction(0.5*L2NormSquared(b=ad2d), A2d)

# Sum two smooth functionals together
objective_function = f1gd  +  f2gd

In [ ]:
# Set algorithm parameters and initialise algorithm
step_size = 0.00002
x0 = ig2d.allocate()

gdbt = GD(x0, objective_function, step_size=None, alpha=1e9, \
          max_iteration = 100000, update_objective_interval = 10)

In [ ]:
gdbt.run(5000, verbose=1)

In [ ]:
plt.figure()
show2D(gdbt.solution, cmap='inferno', fix_range=(vmin,vmax))

In [ ]:
#yline = 89
xline = 107

plt.figure()
plt.plot(recon_fbp2d.subset(horizontal_y=yline).as_array(),':',label='FBP ram-lak')
plt.plot(pdhg.get_output().subset(horizontal_y=yline).as_array(),'-',label='TV PDHG')
plt.plot(gdbt.get_output().subset(horizontal_y=yline).as_array(),'--',label='TV GD')
plt.xlabel('Pixel index')
plt.ylabel('Intensity')
plt.legend()

plt.figure()
plt.plot(recon_fbp2d.subset(horizontal_x=xline).as_array(),':',label='FBP ram-lak')
plt.plot(pdhg.get_output().subset(horizontal_x=xline).as_array(),'-',label='TV PDHG')
plt.plot(gdbt.get_output().subset(horizontal_x=xline).as_array(),'--',label='TV GD')
plt.xlabel('Pixel index')
plt.ylabel('Intensity')
plt.legend()

In [ ]:
# Display unsorted and sorted sinogram

# Unsorted
plt.figure()
show2D(ad2d,cmap='inferno')

# Sort
idx = ad2d.geometry.angles.argsort()

# Sorted
plt.figure()
plt.imshow(ad2d.as_array()[idx,:],cmap='inferno')
plt.xlabel('Detector pixel')
plt.ylabel('Angle')
plt.colorbar(orientation='horizontal')